In [4]:
#%pip install xmltodict
import xmltodict
import json
import os
import xml.etree.ElementTree as ET
import csv

In [32]:
def flatten_xml_to_csv(xml_file: str, csv_file_path: str) -> None:

    xml_payload = ''

    with open(xml_file, 'r', encoding='utf-8') as xf:
        xml_payload = xf.read()

    

    try:
        if xml_payload.index('?>'):
            dec_end = xml_payload.index('?>')+2
            xml_payload = xml_payload[dec_end:]
    except ValueError:
        pass

    print(xml_payload)

    xd = xmltodict.parse(xml_payload)

    def flatten_array(arr: list, parent_key = '', sep = '/'):
        items = []
        for i, val in enumerate(arr):
            if isinstance(val, dict):
                items.extend(flatten_dict(val, f"{parent_key}[{i}]", sep=sep).item())
            elif isinstance(val, list):
                items.extend(flatten_array(val, f"{parent_key}[{i}]", sep=sep))
            else:
                items.append((f"{parent_key}[{i}]", val))

        return items
    
    def flatten_dict(d: dict, parent_key='', sep='/'):
        items = []
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict) and len(v) == 1 and list(v.keys())[0].startswith("@"):
                attr_key = f"{new_key}/{list(v.keys())[0][1:]}"
                items.append((attr_key, list(v.values())[0]))
            elif isinstance(v, dict):
                items.extend(flatten_dict(v, new_key, sep=sep).items())
            elif isinstance(v, list):
                for i, val in enumerate(v):
                    if isinstance(val, dict):
                        items.extend(flatten_dict(val, f"{new_key}[{i}]", sep=sep).items())
                    elif isinstance(val, list):
                        items.extend(flatten_array(val, f"{new_key}[{i}]",sep=sep))
                    else:
                        items.append((f"{new_key}[{i}]",val))
            else:
                items.append((new_key, v))

        return dict(items)
    

    data = flatten_dict(xd)

    if "~" in csv_file_path:
        csv_file_path = os.path.expanduser(csv_file_path)

    print(csv_file_path)

    with open(csv_file_path, 'w', newline='') as f:
        csv_writer = csv.writer(f)

        headers = ["Path", "Value"]
        csv_writer.writerow(headers)

        for k, v in data.items():
            csv_writer.writerow([k,v])
        




In [36]:
x_path = 'states.xml'
c_path = 'states.csv'

flatten_xml_to_csv(x_path, c_path)

x_path = 'states2.xml'
c_path = 'states2.csv'

flatten_xml_to_csv(x_path, c_path)


<states><state><name>California</name><population>39538223</population><city>Los Angeles</city></state><state><name>Texas</name><population>29145505</population><city>Houston</city></state><state><name>Florida</name><population>21538187</population><city>Miami</city></state><state><name>New York</name><population>20201249</population><city>New York City</city></state><state><name>Pennsylvania</name><population>13002700</population><city>Philadelphia</city></state><state><name>Illinois</name><population>12812508</population><city>Chicago</city></state><state><name>Ohio</name><population>11799448</population><city>Columbus</city></state><state><name>Georgia</name><population>10711908</population><city>Atlanta</city></state><state><name>North Carolina</name><population>10439388</population><city>Charlotte</city></state><state><name>Michigan</name><population>10077331</population><city>Detroit</city></state></states>
states.csv

<states><state land_size="423967"><name>California</name><po

In [19]:
def gen_xml_doc_1():
    import xml.etree.ElementTree as ET

    # Create the root element
    root = ET.Element("states")

    # Define the states data
    states_data = [
        {"name": "California", "population": "39538223", "city": "Los Angeles"},
        {"name": "Texas", "population": "29145505", "city": "Houston"},
        {"name": "Florida", "population": "21538187", "city": "Miami"},
        {"name": "New York", "population": "20201249", "city": "New York City"},
        {"name": "Pennsylvania", "population": "13002700", "city": "Philadelphia"},
        {"name": "Illinois", "population": "12812508", "city": "Chicago"},
        {"name": "Ohio", "population": "11799448", "city": "Columbus"},
        {"name": "Georgia", "population": "10711908", "city": "Atlanta"},
        {"name": "North Carolina", "population": "10439388", "city": "Charlotte"},
        {"name": "Michigan", "population": "10077331", "city": "Detroit"}
    ]

    # Populate the XML tree with state data
    for state in states_data:
        state_element = ET.SubElement(root, "state")
        name_element = ET.SubElement(state_element, "name")
        name_element.text = state["name"]
        population_element = ET.SubElement(state_element, "population")
        population_element.text = state["population"]
        city_element = ET.SubElement(state_element, "city")
        city_element.text = state["city"]

    # Create the tree and write to an XML file
    tree = ET.ElementTree(root)
    tree.write("states.xml", encoding="utf-8", xml_declaration=True)


XML file has been generated.


In [35]:
def gen_xml_doc_2():
    import xml.etree.ElementTree as ET

    # Create the root element
    root = ET.Element("states")

    # Define the states data
    states_data = [
        {"name": "California", "population": "39538223", "city": "Los Angeles", "land_size": "423967"},
        {"name": "Texas", "population": "29145505", "city": "Houston", "land_size": "695662"},
        {"name": "Florida", "population": "21538187", "city": "Miami", "land_size": "170312"},
        {"name": "New York", "population": "20201249", "city": "New York City", "land_size": "141297"},
        {"name": "Pennsylvania", "population": "13002700", "city": "Philadelphia", "land_size": "119280"},
        {"name": "Illinois", "population": "12812508", "city": "Chicago", "land_size": "149995"},
        {"name": "Ohio", "population": "11799448", "city": "Columbus", "land_size": "116096"},
        {"name": "Georgia", "population": "10711908", "city": "Atlanta", "land_size": "153910"},
        {"name": "North Carolina", "population": "10439388", "city": "Charlotte", "land_size": "139391"},
        {"name": "Michigan", "population": "10077331", "city": "Detroit", "land_size": "250487"}
    ]

    # Populate the XML tree with state data
    for state in states_data:
        state_element = ET.SubElement(root, "state", land_size=state["land_size"])
        
        name_element = ET.SubElement(state_element, "name")
        name_element.text = state["name"]
        
        population_element = ET.SubElement(state_element, "population")
        population_element.text = state["population"]
        
        city_element = ET.SubElement(state_element, "city")
        city_element.text = state["city"]

    # Create the tree and write to an XML file
    tree = ET.ElementTree(root)
    tree.write("states2.xml", encoding="utf-8", xml_declaration=True)

gen_xml_doc_2()